# LLM Transform Chain

In LangChain, the TransformChain is a specialized chain designed to modify or transform the input data before it's processed by an LLM.

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, TransformChain

In [2]:
yelp_review = open('yelp_review.txt').read()
print(yelp_review)

TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!

REVIEW:
OH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!

FIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.

NOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!

THEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!

THE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIENDLY, AND EXTREMELY KN

### Our Own Custom Transformation Functions

In [3]:
# Take in the input dictionary text and output after any transformations you want

def transformer_fun(inputs: dict) -> dict:
    '''
    Notice how this alwayws takes an input dictionary and also outputs a dictionary. 
    We can call the output and input keys whatever we want, just to make sure to reference it correctly in the chain call.
    '''
    # Grab incoming chain text
    text = inputs['text']
    only_review_text = text.split('REVIEW:')[-1]
    lower_case_text = only_review_text.lower()
    return {'output':lower_case_text}

In [4]:
transform_chain = TransformChain(input_variables=['text'], output_variables=['output'],
                                 transform = transformer_fun)

In [5]:
template = "Create a one sentence summary of this review:\n{review_text}"

In [6]:
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model='gpt-4o')

In [7]:
prompt = ChatPromptTemplate.from_template(template=template)
summary_chain = LLMChain(llm=llm, prompt=prompt, output_key='review_summary')

/var/folders/ny/r3wsv_bd0bs1_vcy4pb5q9pr0000gn/T/ipykernel_17894/3170129382.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summary_chain = LLMChain(llm=llm, prompt=prompt, output_key='review_summary')


In [8]:
sequential_chain = SimpleSequentialChain(chains=[transform_chain, summary_chain], verbose=True)

In [9]:
result = sequential_chain.invoke(yelp_review)



> Entering new SimpleSequentialChain chain...

oh my goodness, where do i begin? this restaurant is absolutely phenomenal! i went there last night with my friends, and we were blown away by the experience!

first of all, the ambiance is out of this world! the moment you step inside, you're greeted with a warm and inviting atmosphere. the decor is stunning, and it immediately sets the tone for an unforgettable dining experience.

now, let's talk about the food! wow, just wow! the menu is a paradise for food lovers. every dish we ordered was a masterpiece. the flavors were bold, vibrant, and exploded in our mouths. from starters to desserts, every bite was pure bliss!

their seafood platter is a must-try! the freshness of the seafood is unmatched, and the presentation is simply stunning. i have never tasted such delicious and perfectly cooked seafood in my life. it's a seafood lover's dream come true!

the service was exemplary. the staff was attentive, friendly, and extremely knowledg

In [10]:
print(result['input'])

TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!

REVIEW:
OH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!

FIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.

NOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!

THEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!

THE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIENDLY, AND EXTREMELY KN

In [11]:
print(result['output'])

This restaurant offers an unforgettable dining experience with stunning ambiance, exquisite food, including a must-try seafood platter, exemplary service, and heavenly desserts, making it a hidden gem worth visiting.


#  LLM Math Chain

In [12]:
from langchain import LLMMathChain

In [14]:
llm_math_model = LLMMathChain.from_llm(llm=llm)
llm_math_model("What is the answer of 17 raised to the power of 11?")

{'question': 'What is the answer of 17 raised to the power of 11?',
 'answer': 'Answer: 34271896307633'}

# Additional Chains

### Connect to Document Vector Store

In [33]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [34]:
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")

In [35]:
db_connection = Chroma(persist_directory='../some_data/US_Constitution/', embedding_function=embedding_function)

### Load QA Models

In [36]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import ChatOpenAI

In [37]:
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model='gpt-4o', temperature=0)

In [38]:
chain = load_qa_chain(llm, chain_type='stuff')
question = "What is 15th ammendment?"

In [39]:
docs = db_connection.similarity_search(question)

In [40]:
print(chain.run(input_documents=docs,question=question))

The 15th Amendment to the United States Constitution, ratified in 1870, prohibits the federal and state governments from denying a citizen the right to vote based on "race, color, or previous condition of servitude." It was one of the Reconstruction Amendments, aimed at ensuring that African American men could exercise their right to vote following the Civil War.


### Get Original Source Context Information

In [41]:
chain = load_qa_with_sources_chain(llm=llm, chain_type='stuff')
query = "What is the 14th amendment?"
docs = db_connection.similarity_search(query)
print(chain.run(input_documents=docs, question=query))

I don't know. 

SOURCES:
